# Load Library & Preprocess Data 

In [1]:
import os
GPU = f'1'
os.environ['CUDA_VISIBLE_DEVICES']=GPU
import random
from glob import glob
import cv2
import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import (
    ReduceLROnPlateau,
    EarlyStopping,
    ModelCheckpoint,
    TensorBoard
)
from tensorflow.keras.applications import (
    MobileNet,
    MobileNetV2,
    EfficientNetB7
)
from tensorflow.keras.layers import (
    GlobalAveragePooling2D,
    Dense,
    Dropout,
    Conv2D
)
from tensorflow.keras import (
    Model,
    Input
)
from sklearn.model_selection import train_test_split

#### Set Seed 

In [2]:
SEED = 42

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

seed_everything(SEED)

#### Config 

In [3]:
batch_size = 64

classes = 1049
size = 256

learning_rate = 1e-2
wd = 0.0005
max_lr = 1e-2
min_lr = 5e-5
cycle_len = 20

EPOCHS = 100

#### Load Data 

In [4]:
df = pd.read_csv('train.csv')
df.head()

,id,landmark_id
0,../../uiui/datasets/public/train/경기도/동탄 복합문화센터...,114
1,../../uiui/datasets/public/train/경기도/동탄 복합문화센터...,114
2,../../uiui/datasets/public/train/경기도/동탄 복합문화센터...,114
3,../../uiui/datasets/public/train/경기도/동탄 복합문화센터...,114
4,../../uiui/datasets/public/train/경기도/동탄 복합문화센터...,114


In [5]:
df['landmark_id'] = df['landmark_id'].astype(str)

#### Split Train & Valid 

In [6]:
train_df, val_df = train_test_split(df, test_size=0.2, random_state=SEED)

In [7]:
train_df.shape, val_df.shape

((70481, 2), (17621, 2))

# Datagenerator 

In [8]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
)
val_datagen = ImageDataGenerator(
    rescale=1./255,
)

train_generator = train_datagen.flow_from_dataframe(
    train_df,
    x_col='id',
    y_col='landmark_id',
    target_size=(size,size),
    batch_size=batch_size,
)
val_generator = val_datagen.flow_from_dataframe(
    val_df,
    x_col='id',
    y_col='landmark_id',
    target_size=(size,size),
    batch_size=batch_size,
)

Found 70481 validated image filenames belonging to 1049 classes.
Found 17621 validated image filenames belonging to 1049 classes.


# Build Model 

In [9]:
def BaseModel():
    x = inputs = Input([size, size, 3])
    x = Conv2D(64, 3, activation='relu')(x)
    x = Conv2D(64, 3, activation='relu')(x)
    x = Conv2D(64, 3, activation='relu')(x)
    x = GlobalAveragePooling2D()(x)
    output = Dense(classes, activation='softmax')(x)
    return Model(inputs=inputs, outputs=output)

with tf.device(f'/device:GPU:{GPU}'):
    model = BaseModel()

model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 254, 254, 64)      1792      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 252, 252, 64)      36928     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 250, 250, 64)      36928     
_________________________________________________________________
global_average_pooling2d (Gl (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 1049)              68185     
Total params: 143,833
Trainable params: 143,833
Non-trainable params: 0
________________________________________________

# Compile 

In [10]:
optimizer = tfa.optimizers.AdamW(learning_rate, wd)

model.compile(
    optimizer = optimizer,
    loss = tf.keras.losses.CategoricalCrossentropy(),
    metrics = ['accuracy']
)

#### callbacks 

In [11]:
checkpoint_filepath = './checkpoints/base_model.h5'
mon = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath, save_weights_only=False,
                                                monitor='val_accuracy', mode='max', save_best_only=True, verbose=1)

def trianfle_fn(x):
    return 1. / (2.**(x - 1))
clr_f = tfa.optimizers.CyclicalLearningRate(
    initial_learning_rate = max_lr,
    maximal_learning_rate = min_lr,
    step_size = cycle_len,
    scale_fn = trianfle_fn
)

clr = tf.keras.callbacks.LearningRateScheduler(clr_f)
es = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, patience=30, 
                                          verbose=2, mode='auto', baseline=None, 
                                          restore_best_weights=True)

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="./logs")

callbacks = [mon, clr, es, tensorboard_callback]
# callbacks = [mon, clr]
# callbacks = [clr, es]
# callbacks = [clr]
# callbacks = []

# Training 

In [ ]:
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    callbacks=callbacks,
    validation_data=val_generator,
)

Epoch 1/100
   1/1102 [..............................] - ETA: 0s - loss: 6.9576 - accuracy: 0.0000e+00WARNING:tensorflow:From /home/ubuntu/anaconda3/envs/Landmark/lib/python3.7/site-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
 682/1102 [=================>............] - ETA: 5:01 - loss: 6.9594 - accuracy: 7.1099e-04